# <hr style="clear: both" />

# Running Spark in YARN-client mode

This notebook demonstrates how to set up a SparkContext that uses SURFsara's Hadoop cluster: [YARN resourcemanager](http://head05.hathi.surfsara.nl:8088/cluster) (note you will need to be authenticated via kerberos on your machine to visit the resourcemanager link) for executors.

First initialize kerberos via a Jupyter terminal. 
In the terminal execute: <BR>
<i>kinit -k -t data/robertop.keytab robertop@CUA.SURFSARA.NL</i><BR>
Print your credentials:


In [1]:
! klist

Ticket cache: FILE:/tmp/krb5cc_1001
Default principal: robertop@CUA.SURFSARA.NL

Valid starting       Expires              Service principal
04/12/2016 13:42:29  04/13/2016 13:42:29  krbtgt/CUA.SURFSARA.NL@CUA.SURFSARA.NL
	renew until 04/12/2016 13:42:29


In [4]:
! hdfs dfs -ls 

Verify that we can browse HDFS:

Next initialize Spark. Note that the code below starts a job on the Hadoop cluster that will remain running while the notebook is active. Please close and halt the notebook when you are done. Starting the SparkContext can take a little longer. You can check the YARN resourcemanager to see the current status/usage of the cluster.

In [ ]:
import os
os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python2.7'

HDFS_PATH = "hdfs://hathi-surfsara"

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sconf = SparkConf()

# Master is now yarn-client. The YARN and hadoop config is read from the environment
sconf.setMaster("yarn-client")

# You can control many Spark settings via the SparkConf. This determines the amount of executors on the cluster:
sconf.set("spark.executor.instances", "400")
#sconf.set("spark.executor.memory", "2g")

# UFW (firewall) is active on the VM. We explicitly opened these ports and Spark should not bind to random ports:
sconf.set("spark.driver.port", 51800)
sconf.set("spark.fileserver.port", 51801)
sconf.set("spark.broadcast.port", 51802)
sconf.set("spark.replClassServer.port", 51803)
sconf.set("spark.blockManager.port", 51804)
sconf.set("spark.authenticate", True)
sconf.set("spark.yarn.keytab", "/home/jovyan/work/data/robertop.keytab")
sconf.set("spark.yarn.access.namenodes", HDFS_PATH + ":8020")

sc = SparkContext(conf=sconf)

sqlCtx = SQLContext(sc) 

# <hr style="clear: both" />

# Now you can run your code

Pick a clustering algorithm (name of the file that provides a classify(x,y [,threshold]) function)

In [ ]:
execfile('../spark-scripts/conventions.py')
execfile('../spark-scripts/splitCluster.py')
#execfile('../spark-scripts/utils.py')
execfile('../spark-scripts/eval.py')
execfile('../spark-scripts/implicitPlaylistAlgoFunctions.py')
execfile('../spark-scripts/implicitPlaylistAlgoMain.py')

CLUSTER_ALGO = 'jaccardBase'

execfile('../spark-scripts/' + CLUSTER_ALGO + '.py')


# Reading the conf file

In [ ]:
import json
import copy



conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = 'test'
conf['split']['split'] = conf['split']['name']
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = HDFS_PATH + '/user/robertop/mattia/clusterBase.split/SenzaRipetizioni_1'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = HDFS_PATH + '/user/robertop/mattia'
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 1
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False
conf['split']["prop"] = {'reclistSize': conf['split']['reclistSize']}
conf['split']['type'] = list
conf['split']['out'] = HDFS_PATH + '/user/robertop/mattia/clusterBase.split/'
conf['split']['location'] = '30Mdataset/relations/sessions'

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = "clusterBase.split"
conf['general']['bucketName'] = 'head02.hathi.surfsara.nl/user/robertop/mattia'
conf['general']['tracksPath'] = '30Mdataset/entities/tracks.idomaar.gz'

conf['algo'] = {}
conf['algo']['name'] = 'ClusterBase'
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']["sessionJaccardShrinkage"] = 5
conf['algo']['props']["clusterSimilarityThreshold"] = 0.1
conf['algo']['props']["expDecayFactor"] = 0.7
# ****** END EXAMPLE ****************#


Pick the list of songs ad create clusters


In [ ]:
import json
import string

def my_replace_punct(x):
    ret = ""
    for i in x:
        if i == '+':
            ret += ' '
        else:
            ret += i
    return ret

tracksRDD = sc.textFile(HDFS_PATH + '/user/robertop/mattia/30Mdataset/entities/tracks.idomaar.gz')
tracksRDD = tracksRDD.map(lambda x: x.split('\t')).map(lambda x: (x[1], json.loads(x[3])['name'].split('/') ) )
tracksRDD = tracksRDD.map(lambda x: (x[0], " ".join( (x[1][0], x[1][2]) ) ))
tracksRDD = tracksRDD.map(lambda x : (x[0], my_replace_punct(x[1])))
tracksRDD = tracksRDD.map(lambda x: (x[0], tokenize_song(x[1]), x[1]))

sampleRDD = tracksRDD.take(5000)
sampleRDD = sc.parallelize(sampleRDD)
sampleRDD.take(3)

In [ ]:
#Build an RDD with ('word' -> (id, name))
wordsRDD = sampleRDD.flatMap(lambda x: [(i, (x[0], x[2])) for i in x[1]] )
wordsRDD.take(3)

#Group by 'word' and keep only the ones with more then 1 song
wordsRDD = wordsRDD.groupByKey().mapValues(list).filter(lambda x: len(x[1]) > 1)

#Compute a cartesian product for each list of songs with a common word
def filtered_cartesian(x):
    equal_couples = []
    for i in range(len(x[1])):
        a = x[1][i]
        id_a = x[1][i][0]
        name_a = x[1][i][1]
        
        for j in range(i):
            b = x[1][j]
            id_b = x[1][j][0]
            name_b = x[1][j][1]
            if id_a != id_b:
                if classify(name_a, name_b):
                    equal_couples.append((a,b))
                    
    return (x[0], equal_couples)

coupleRDD = wordsRDD.map(filtered_cartesian).filter(lambda x: len(x[1]) > 1)
coupleRDD.take(10)





In [ ]:


#Merge every fucking couple with common songs
def couple_merge(x):
    result = []
    for current in x[1]:
        merged = False
        for cluster in result:
            if len(set(current) & set(cluster)):
                result.append(tuple(set(current) | set(cluster)))
                merged = True
        if not merged:
            result.append(current)
    return (x[0], result)

mergedRDD = coupleRDD.map(couple_merge)

#Flat the result and take the distincts
#coupleRDD = coupleRDD.flatMap(lambda x: (i for i in x)).distinct()

mergedRDD.take(10)

In [ ]:
splitter(conf)

In [ ]:
from os import path
basePath = path.join("hdfs://", conf['general']['bucketName'], conf['general']['clientname'])
splitPath = path.join(basePath, conf['split']['name'])

clusterSimList = [0.1]
sessionJaccardShrinkageList = [5]
expDecayList = [0.7]

for exclude in [True]:
    conf['split']['excludeAlreadyListenedTest'] = str(exclude)
    #conf['split']['name'] = 'giroCompletoTestMultipleConfs_exclude%s' % exclude
    #splitter(conf)
    train, test = loadDataset(conf)
    train.cache()
    test.cache()
    
    for sessionJaccardShrinkage in sessionJaccardShrinkageList:
        conf['algo']['props']["sessionJaccardShrinkage"] = sessionJaccardShrinkage
        
        for clusterSim in clusterSimList:
            conf['algo']['props']["clusterSimilarityThreshold"] = clusterSim
            
            playlists = extractImplicitPlaylists(train, conf).cache()
            
            for expDecay in expDecayList:
                conf['algo']['props']["expDecayFactor"] = expDecay
                conf['algo']['name'] = 'ImplicitPlaylist_shk_%d_clustSim_%.3f_decay_%.3f' % \
                    (sessionJaccardShrinkage, clusterSim, expDecay )

                recJsonRDD = executeImplicitPlaylistAlgo(playlists, test, conf)
                try:
                    saveRecommendations(conf, recJsonRDD, overwrite=True)
                    try:
                        computeMetrics(conf)
                    except:
                        print 'Error in computing metrics'
                except:
                    print 'Error in saving recommndations'
                    try:
                        computeMetrics(conf)
                    except:
                        print 'Error in computing metrics'